### Load Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt
import math
import seaborn as sns

# Import Custom Module
import OeconToolbox as ott

%load_ext autoreload
%autoreload 2
%matplotlib inline

### [OPTIONAL] Set Matplotlib Style

In [3]:
matplotlib.style.use('default')

matplotlib.rcParams.update({
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 10})
matplotlib.path.should_simplify = True

sns.set_style("ticks")

cm = 1/2.54
textwidth = 13.998 / 2.54

## Import Cumulative Default Table

In [5]:
DefTable = pd.read_excel("CumulativeDefaultTable.xlsx",header=[0],index_col=[0],skipfooter=5,usecols="A:K")
DefTable.round(2)

,1,2,3,4,5,6,7,8,9,10
AAA,0.00,0.03,0.10,0.19,0.27,0.37,0.41,0.46,0.52,0.58
AA,0.02,0.06,0.11,0.21,0.30,0.41,0.49,0.56,0.63,0.70
A,0.05,0.13,0.22,0.33,0.46,0.60,0.76,0.90,1.05,1.20
BBB,0.16,0.43,0.75,1.14,1.54,1.94,2.27,2.61,2.94,3.24
BB,0.63,1.93,3.46,4.99,6.43,7.75,8.89,9.90,10.82,11.64
B,3.34,7.80,11.75,14.89,17.35,19.36,20.99,22.31,23.50,24.62


# Simulating a Portfolio of loans
The SPV portfolio consists of $J$ loans from identical corporate issuers with a single class of debt outstanding. The parameters of the issuers is identical to those described in the 'Collateral Dynamics.ipynb' notebook.

## Parameters of the Corporate Issuers

In [1]:
V0 = 100      # Initial Asset Value
rf = .035     # Risk-Free Rate
rm = .105     # Market Return
beta = .8     # Firm CAPM Coefficient
sigma_i = .25 # Idiosyncratic Risk
sigma_m = .14 # Market Risk
ttm = 5       # Time to Maturity of Debt

Auxiliary Calculations

In [6]:
mu = rf + beta * (rm - rf)
sigma = ott.sigma_beta_adj(beta, sigma_m, sigma_i)
face_value = ott.facevalue_from_probability(DefTable.loc["B", ttm] / 100, V0, ttm, mu, sigma)
market_value = ott.mv_bond(V0, face_value, ttm, rf, sigma)

## Parameters of the SPV

In [18]:
J = 125    # number of loans
N = 250000 # Number of simulated portfolios

# Modelling the SPV Dynamics without Prepayment on Collateral Assets

In [22]:
max_spv_cash_flows = face_value * J
initial_market_value = market_value * J
print("The absolute maximum payoff: {0:0.2f} with initial market value: {1:0.2f}".format(max_spv_cash_flows,initial_market_value))

The absolute maximum payoff: 9180.30 with initial market value: 6990.38


A wrapper function for the equilibrium value of the loan portfolio

In [16]:
def Dk(face_val):
    Dk, M = ott.loan_portfolio(j = J,
                               n = N,
                               V = V0,
                               B = face_value,
                               T = ttm,
                               rf = rf,
                               rm = rm,
                               beta = beta,
                               sigmaI = sigma_i,
                               sigmaM = sigma_m,
                               risk_neutral=True,
                               paths=False)
    return Dk

Total SPV payoffs (under Q-measure)

In [20]:
SPV_Q = Dk(face_value)

Total SPV payoffs (under P-measure)

In [21]:
SPV_P, M = ott.loan_portfolio(j = J,
                              n = 250000,
                              V = V0,
                              B = face_value,
                              T = ttm,
                              rf = rf,
                              rm = rm,
                              beta = beta,
                              sigmaI = sigma_i,
                              sigmaM = sigma_m,
                              risk_neutral=False,
                              paths=False)